In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ABES KAGGLE/ABES_KAGGLE_2/train_Kaggle_Comp.csv')

In [ ]:
df1.head()

,employee_id,age,gender,education_level,experience_years,job_role,company_type,working_hours_per_day,projects_handled,overtime_hours,...,remote_work,team_size,manager_support,job_satisfaction,mental_fatigue_score,stress_level,sleep_hours,physical_activity,health_issues,burnout_score
0,10001,50,Male,Bachelor,0,Data Analyst,MNC,7,3,24,...,Yes,7,1,5,9.9,4,5.8,No,No,0.627
1,10002,36,Male,Bachelor,6,Data Analyst,Corporate,8,5,27,...,No,14,1,2,8.9,4,7.2,Yes,Yes,0.586
2,10003,29,Female,Bachelor,25,Project Manager,Startup,9,6,7,...,Yes,9,3,3,7.1,5,6.9,Yes,Yes,0.874
3,10004,42,Female,Master,6,HR Executive,Startup,8,5,19,...,No,15,4,5,7.9,3,4.8,Yes,Yes,0.507
4,10005,40,Male,Master,6,Data Analyst,Corporate,7,3,26,...,Yes,17,4,5,4.2,3,7.6,Yes,No,0.449


In [ ]:
df2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ABES KAGGLE/ABES_KAGGLE_2/test_Kaggle_Comp.csv')

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 28 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   age                         4000 non-null   int64  
 1   experience_years            4000 non-null   int64  
 2   working_hours_per_day       4000 non-null   int64  
 3   projects_handled            4000 non-null   int64  
 4   overtime_hours              4000 non-null   int64  
 5   work_pressure_score         4000 non-null   int64  
 6   work_life_balance           4000 non-null   int64  
 7   team_size                   4000 non-null   int64  
 8   manager_support             4000 non-null   int64  
 9   job_satisfaction            4000 non-null   int64  
 10  mental_fatigue_score        4000 non-null   float64
 11  stress_level                4000 non-null   int64  
 12  sleep_hours                 4000 non-null   float64
 13  burnout_score               4000 

In [ ]:

# automatically select categorical columns
cat_cols = df1.select_dtypes(include=['object', 'category']).columns

# apply one-hot encoding to all of them at once
df1 = pd.get_dummies(df1, columns=cat_cols, drop_first=True)


In [ ]:
df1 = df1.drop(columns=['employee_id'])


In [ ]:
df1.isnull().values.any()

np.False_

#ElasticNet+XgBoost

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNetCV

In [ ]:
X=df1.drop(['burnout_score'],axis=1)
Y=df1['burnout_score']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = train_test_split(
    X, Y, test_size=0.2, random_state=42
)


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)


In [ ]:
from sklearn.linear_model import ElasticNetCV

enet = ElasticNetCV(
    l1_ratio=[0.1,0.3,0.5,0.7,0.9,1.0],
    n_alphas=100,
    cv=5,
    max_iter=5000,
    random_state=42
)

enet.fit(X_train_scaled, Y_train)


ElasticNetCV(cv=5, l1_ratio=[0.1, 0.3, 0.5, 0.7, 0.9, 1.0], max_iter=5000,
             random_state=42)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

Y_pred = enet.predict(X_val_scaled)

print("R² Score:", r2_score(Y_val, Y_pred))
print("RMSE:", np.sqrt(mean_squared_error(Y_val, Y_pred)))


R² Score: 0.9436629022268163
RMSE: 0.06233412568256383


In [ ]:
# best ElasticNet hyperparameters
print("Best alpha:", enet.alpha_)
print("Best l1_ratio:", enet.l1_ratio_)

# create coefficient series with correct feature names
coef_series = pd.Series(enet.coef_, index=X.columns)

print("\nCoefficients:\n", coef_series)

# select non-zero coefficient features
selected_features = coef_series[coef_series != 0].index.tolist()

print("\nSelected features:", selected_features)


Best alpha: 0.0010653048455097668
Best l1_ratio: 1.0

Coefficients:
 age                           0.000000
experience_years             -0.000000
working_hours_per_day         0.061954
projects_handled             -0.000000
overtime_hours               -0.000725
work_pressure_score           0.134137
work_life_balance            -0.064905
team_size                     0.000000
manager_support              -0.038267
job_satisfaction             -0.052267
mental_fatigue_score          0.162862
stress_level                  0.063326
sleep_hours                  -0.041298
gender_Male                  -0.000000
education_level_Master        0.000375
education_level_PhD           0.000000
job_role_HR Executive         0.000000
job_role_ML Engineer         -0.000000
job_role_Project Manager      0.000284
job_role_Software Engineer   -0.000467
company_type_MNC              0.000000
company_type_Startup         -0.000000
deadline_frequency_Low       -0.000000
deadline_frequency_Medium    -0.00

In [ ]:
X_train_sel = X_train[selected_features]
X_val_sel   = X_val[selected_features]


In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

xgb_base = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='reg:squarederror',
    random_state=42
)

xgb_base.fit(X_train_sel, Y_train)

Y_pred_base = xgb_base.predict(X_val_sel)

print("Baseline R²:", r2_score(Y_val, Y_pred_base))
print("Baseline RMSE:", np.sqrt(mean_squared_error(Y_val, Y_pred_base)))


Baseline R²: 0.9387926822342347
Baseline RMSE: 0.0649726097252827


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

param_dist = {
    'n_estimators': np.arange(300, 1200, 100),
    'max_depth': np.arange(3, 10),
    'learning_rate': np.linspace(0.01, 0.1, 10),
    'subsample': np.linspace(0.6, 1.0, 5),
    'colsample_bytree': np.linspace(0.6, 1.0, 5),
    'min_child_weight': [1, 3, 5, 7],
    'gamma': [0, 0.1, 0.2, 0.3]
}

xgb = XGBRegressor(
    objective='reg:squarederror',
    random_state=42
)

random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=80,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

random_search.fit(X_train_sel, Y_train)


Fitting 5 folds for each of 80 candidates, totalling 400 fits


RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          feature_weights=None, gamma=None,
                                          grow_policy=None,
                                          importance_type=None,
                                          interaction_constraint...
                   param_distributions={'colsample_bytree': array([0.6, 0.7, 0.8, 0.9, 1. ]),
                                        'gamma': [0, 0.1, 0.2, 0.3],
                                        'learning_rate': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ]),
                                        'max_depth': array([3, 4, 5, 6, 7, 8, 9]),
                                        'min_child_weight': [1, 3, 5, 7],
                                        'n_estimators': array([ 300,  400,  500,  600,  700,  800,  900, 1000, 1100]),
                                        'subsample': array([0.6, 0.7, 0.8, 0.9, 1. ])},
                   random_state=42, scoring='r2', verbose=1)

In [ ]:
best_params_random = random_search.best_params_
best_params_random


{'subsample': np.float64(0.6),
 'n_estimators': np.int64(700),
 'min_child_weight': 5,
 'max_depth': np.int64(3),
 'learning_rate': np.float64(0.04000000000000001),
 'gamma': 0,
 'colsample_bytree': np.float64(0.6)}

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [
        best_params_random['n_estimators'] - 100,
        best_params_random['n_estimators'],
        best_params_random['n_estimators'] + 100
    ],
    'max_depth': [
        best_params_random['max_depth'] - 1,
        best_params_random['max_depth'],
        best_params_random['max_depth'] + 1
    ],
    'learning_rate': [
        best_params_random['learning_rate'] - 0.01,
        best_params_random['learning_rate'],
        best_params_random['learning_rate'] + 0.01
    ],
    'subsample': [best_params_random['subsample']],
    'colsample_bytree': [best_params_random['colsample_bytree']],
    'min_child_weight': [best_params_random['min_child_weight']],
    'gamma': [best_params_random['gamma']]
}

grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train_sel, Y_train)


Fitting 5 folds for each of 27 candidates, totalling 135 fits


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, feature_weights=None,
                                    gamma=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None...
             param_grid={'colsample_bytree': [np.float64(0.6)], 'gamma': [0],
                         'learning_rate': [np.float64(0.030000000000000006),
                                           np.float64(0.04000000000000001),
                                           np.float64(0.05000000000000001)],
                         'max_depth': [np.int64(2), np.int64(3), np.int64(4)],
                         'min_child_weight': [5],
                         'n_estimators': [np.int64(600), np.int64(700),
                                          np.int64(800)],
                         'subsample': [np.float64(0.6)]},
             scoring='r2', verbose=1)

In [ ]:
best_xgb = grid_search.best_estimator_

Y_pred_best = best_xgb.predict(X_val_sel)

print("🔥 Final XGBoost R²:", r2_score(Y_val, Y_pred_best))
print("🔥 Final XGBoost RMSE:", np.sqrt(mean_squared_error(Y_val, Y_pred_best)))


🔥 Final XGBoost R²: 0.9418529182157925
🔥 Final XGBoost RMSE: 0.06332753724403394


In [ ]:
# find missing columns in test
missing_cols = set(selected_features) - set(df2.columns)

print("Missing columns in test:", missing_cols)

# add them with 0 values
for col in missing_cols:
    df2[col] = 0


Missing columns in test: {'education_level_Master', 'job_role_Project Manager', 'job_role_Software Engineer', 'physical_activity_Yes'}


In [ ]:
df2.head()

,employee_id,age,gender,education_level,experience_years,job_role,company_type,working_hours_per_day,projects_handled,overtime_hours,...,work_life_balance,remote_work,team_size,manager_support,job_satisfaction,mental_fatigue_score,stress_level,sleep_hours,physical_activity,health_issues
0,14001,31,Female,PhD,12,HR Executive,Corporate,9,4,12,...,2,No,14,2,1,6.8,1,6.1,No,No
1,14002,47,Male,Bachelor,10,HR Executive,Corporate,10,1,15,...,5,Yes,15,1,1,7.0,4,6.3,Yes,No
2,14003,28,Male,PhD,33,HR Executive,Startup,8,1,20,...,5,Yes,11,5,1,6.3,1,5.0,Yes,No
3,14004,28,Male,PhD,28,Data Analyst,Corporate,11,6,24,...,5,Yes,18,4,3,8.7,5,5.5,Yes,Yes
4,14005,36,Female,Bachelor,11,ML Engineer,MNC,6,6,14,...,5,Yes,3,3,2,7.5,5,4.9,No,No


In [ ]:
if 'employee_id' in df2.columns:
    test_ids = df2['employee_id']
    df2 = df2.drop(columns=['employee_id'])
else:
    test_ids = None


In [ ]:
X_test_sel = df2[selected_features]


In [ ]:
Y_test_pred = best_xgb.predict(X_test_sel)


In [ ]:
submission = pd.DataFrame({
    'employee_id': test_ids,
    'burnout_score': Y_test_pred
})

submission.to_csv("/content/drive/MyDrive/Colab Notebooks/ABES KAGGLE/ABES_KAGGLE_2/submission.csv", index=False)


In [ ]:
X_test_sel.shape
submission.head()
submission.isnull().sum()


,0
employee_id,0
burnout_score,0
